# TP MODELISATION DU CHURN

# Sommaire

1. [Import de la table préparée et Séparation Train / Test](#sect1)
2. [Standardisation des variables](#sect2)
3. [Modélisation](#sectmodel)
  - [Régression pénalisée](#sect41)
  - [Random Forest](#sect43)
  - [GBM (Gradient Boosting Machine)](#sect44)

In [ ]:
import pandas as pd
import os
import numpy as np
import pickle
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,f1_score,roc_auc_score,roc_curve,auc

import matplotlib.pyplot as plt

# 1 Chargement des données préparées <a name="sect1" ></a>

In [ ]:
repertoire = "../data"
os.chdir(repertoire)
file=open("churn_prepared.pydata","rb")
data=pickle.load(file)
file.close()

In [ ]:
data.shape

In [ ]:
data.describe()


In [ ]:
data.dtypes

In [ ]:
X= data.drop('Churn?',axis=1)
y=data['Churn?'].astype(int)

In [ ]:
y.value_counts(normalize=True)

In [ ]:
# split Apprentissage Test
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X,y, stratify=y, test_size=0.3,random_state=42)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
y_train.sum(), y_test.sum()

In [ ]:
y_train.mean(), y_test.mean()

# 2. Standardisation des variables <a name="sect2" ></a>

In [ ]:
from sklearn.preprocessing import StandardScaler
norm=StandardScaler().fit(X_train) 
X_train = pd.DataFrame(norm.transform(X_train), columns=X.columns)
X_test = pd.DataFrame(norm.transform(X_test), columns=X.columns)

In [ ]:
X_train.describe()

In [ ]:
X_test.describe()

# 3. Modélisation <a name="sectmodel" ></a>

In [ ]:
# Import de la fonction de lift à utiliser
os.chdir('../modules/')
from fonctions_metrics import lift

## 3.1 PARAMETRER LE MODELE REG PENALISEE <a name="sect41" ></a>
Choisir ridge (L2) ou lasso (L1) ou elasticnet, avec la classe LogisticRegression
Définir une grille d'hyper param pour C
et utliser GridSearchCV pour la balayer

In [ ]:
## Regression logistique avec pénalité lasso et grid search
#### on cherche par CV le meilleur C (1/alpha) le coef de regularisation

In [ ]:
param = [{"C":[0.001,0.005,0.01,0.025,0.05,0.075,0.1], "class_weight":["balanced",None]}]
# Instancie le modele LASSO : construction classe
lr=LogisticRegression(penalty='l1',solver='liblinear')
# Grid Search : on indique le modele et la grille de param
modeleLassoCV = GridSearchCV (lr,param,cv = 4,n_jobs=-1,scoring='roc_auc', refit=True)
modeleLassoCV = modeleLassoCV.fit(X_train,y_train)
modeleLassoCV.best_params_

In [ ]:
modeleLassoCV.best_params_['C']

In [ ]:
pd.DataFrame(modeleLassoCV.cv_results_).sort_values(['rank_test_score'])

In [ ]:
# appliquer la méthode fit() : apprend sur les données de TRAIN
modeleLassoCV.fit(X_train,y_train)

In [ ]:
# Visualisation des coefficients estimées pour chaque variable
coef=list(modeleLassoCV.best_estimator_.coef_[0])
coef_df = pd.DataFrame({'Coefficients': list(coef)}, list(X_train.columns.values))
coef_df.sort_values(['Coefficients'], ascending=False)

In [ ]:
# combien d'élemnts non nuls ?
feature_0= [i for i in coef if i==0] #print(feature_0)
print(len(feature_0))
# PLus on baisse C, plus de colonnes s'annulent (0)

### Analyser les résultats et les performances 

In [ ]:
# Prediction des probabilités de 1 avec 
# Colonne 1 signifie la seconde modalité du vecteur soit la proba Churn=1
probas_test = modeleLassoCV.predict_proba(X_test)[:,1]
probas_train = modeleLassoCV.predict_proba(X_train)[:,1]

In [ ]:
# score: accuracy taux de bien classé global au cutoff de 0.5
print(modeleLassoCV.score(X_test, y_test))
print(modeleLassoCV.score(X_train,y_train))

In [ ]:
#AUC
print(roc_auc_score(y_train,probas_train))
print(roc_auc_score(y_test,probas_test))

In [ ]:
print(lift(probas_train,X_train,y_train))
print(lift(probas_test,X_test,y_test))

# Le lift à 10% : je prend les 10% plus fortes proba de churn (individus les plus risqués),
# taux de churn / taux de churn moyen 

In [ ]:
#compute lift at 5%
print(lift(probas_train,X_train,y_train,p=5))
print(lift(probas_test,X_test,y_test,p=5))

In [ ]:
# ENR LES RESULTATS DANS UN DICTIONNAIRE POUR POUVOIR COMPARER
model='Score Lasso'
# métriques (liste de dictionnaires)
metriques = [{'model':model,'AUC_test':round(roc_auc_score(y_test,probas_test),3),'lift at 5':lift(probas_test,X_test,y_test,p=5),'lift at 10':lift(probas_test,X_test,y_test,p=10)}]
metriques

In [ ]:
# mat de confusion à un cutoff donné
target_names = ['Fidèles','Churners']

# métriques au cutoff donné
print(classification_report(y_train,modeleLassoCV.predict(X_train), target_names=target_names))
# métriques au cutoff par défaut
print(classification_report(y_test,modeleLassoCV.predict(X_test), target_names=target_names))



In [ ]:
from fonctions_metrics import CAP_table, auc_et_roc
CAP_table(pd.Series(probas_test, index= y_test.index),y_test, stepsize=5,n=10 )

In [ ]:
auc_et_roc(y_train, probas_train)

In [ ]:
auc_et_roc(y_test, probas_test)

#### Sauvegarde du modèle en pickle

In [ ]:
with open("../data/my_churn_model.pkl", "wb") as file:
    pickle.dump(modeleLassoCV, file)

In [ ]:
# Test de réapplication
with open("../data/my_churn_model.pkl", "rb") as file:
    test_reapp_model = pickle.load(file)

In [ ]:
test_reapp_model.predict_proba(X_test.head(5))

## 3.2 PARAMETRER LE MODELE RANDOMFOREST <a name="sect43" ></a>
Tester un modle light peu profond et un modele complexe

In [ ]:
# exemple randomforest avec des rbres peu profonds
rf_light = RandomForestClassifier(                                                                       
                                       n_jobs = -1, # coeurs
                                       random_state = 42
                                     )    

param = [{"n_estimators":[100,200,300], "class_weight":["balanced",None], "max_depth":[3,4,5],"min_samples_split":[0.05,0.1]}]

# Grid Search : on indique le modele et la grille de param
modelerfCV = GridSearchCV (rf_light,param,cv = 4,n_jobs=-1,scoring='roc_auc', refit=True)
modelerfCV = modelerfCV.fit(X_train,y_train)
modelerfCV.best_params_

In [ ]:
pd.DataFrame(modelerfCV.cv_results_).sort_values(['rank_test_score'])

In [ ]:
# Importance des variables Light
df=pd.DataFrame(modelerfCV.best_estimator_.feature_importances_,X_train.columns.values)
df.columns=['Importance']
df.sort_values(by='Importance',ascending=False)

### Analyser les résultats et les performances (light)

In [ ]:
# Prediction des probabilités de 1 ( array2d ) avec fonction predict_proba
probas_test = modelerfCV.predict_proba(X_test)[:,1]
probas_train = modelerfCV.predict_proba(X_train)[:,1]

In [ ]:
#AUC sur train et test avec roc_auc_score
print(roc_auc_score(y_train,probas_train))
print(roc_auc_score(y_test,probas_test))
# un peu de surapprentissage mais peu ce qui est normal car ce sont des arbres peu profonds

In [ ]:
print(lift(probas_train,X_train,y_train))
print(lift(probas_test,X_test,y_test))

In [ ]:
print(classification_report(y_train,modelerfCV.predict(X_train), target_names=target_names))
print(classification_report(y_test,modelerfCV.predict(X_test), target_names=target_names))

In [ ]:
# AJOUTE LES RESULTATS
model='rf'
# métriques (liste de dictionnaires)
metriques.append([{'model':model,'AUC_test':round(roc_auc_score(y_test,probas_test),3),'lift at 5':lift(probas_test,X_test,y_test,p=5),'lift at 10':lift(probas_test,X_test,y_test,p=10)}])
metriques

## 3.4 PARAMETRER LE MODELE GRADIENT BOOSTING MACHINE <a name="sect44" ></a>


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
gbt_noRand05=GradientBoostingClassifier(
                                       random_state=42)

param = [{"n_estimators":[100,200,300], "max_depth":[2,3],"subsample":[0.9,1.0], "learning_rate":[0.001,0.01,0.05]}]

# Grid Search : on indique le modele et la grille de param
modelegbmCV = GridSearchCV (gbt_noRand05,param,cv = 4,n_jobs=-1,scoring='roc_auc', refit=True)
modelegbmCV = modelegbmCV.fit(X_train,y_train)
modelegbmCV.best_params_

In [ ]:
pd.DataFrame(modelegbmCV.cv_results_).sort_values(['rank_test_score'])

Analyse du nb iterations optimal en affichant la fonction de perte sur le test

In [ ]:
from sklearn.metrics import log_loss

niter = modelegbmCV.best_estimator_.n_estimators
iter = np.arange(niter) + 1
test_deviance = np.zeros((niter,), dtype=np.float64)

# staged_predict_proba donne les probabilités à chaque itération
for i, y_pred_proba in enumerate(modelegbmCV.best_estimator_.staged_predict_proba(X_test)):
    test_deviance[i] = log_loss(y_test, y_pred_proba)

plt.figure(figsize=(8, 6))
plt.plot(iter, test_deviance, label='Test', color='darkorange')
plt.plot(iter, modelegbmCV.best_estimator_.train_score_, label='Apprentissage', color='navy')
plt.legend(loc="upper right", fontsize=12)
plt.show()


In [ ]:
# Algo avc les mêmes params sauf itértionq qu'on met à 100
gbt_Rand=GradientBoostingClassifier(**modelegbmCV.best_params_)
gbt_Rand.n_estimators = 100
# Apprentissage du modele
gbt_Rand.fit(X_train,y_train)
gbt_Rand

In [ ]:
niter=gbt_Rand.n_estimators
iter = np.arange(niter) + 1
test_deviance = np.zeros((niter,), dtype=np.float64)
# staged_decision_functio : décision fonction à chaque iteration
for i, y_pred_proba in enumerate(gbt_Rand.staged_predict_proba(X_test)):
    test_deviance[i] = log_loss(y_test, y_pred_proba)

plt.figure(figsize=(8, 6))
# Erreur sur le test (evolution deviance)
plt.plot(iter,test_deviance,label='Test',color='darkorange')
        # min vers 100 
# Erreur sur apprentissage (evolution deviance)
plt.plot(iter,gbt_Rand.train_score_,label='Apprentissage',color='navy')    
# Diminution de l'erreur rapport modele precedant (par rapport au oob)
#plt.plot(iter,gbt_noRand05.oob_improvement_)
plt.legend(loc="upper right", fontsize=12)

### Analyser les résultats et les performances (light) 

In [ ]:
# Prediction des probabilités de 1 , array2d
probas_test = modelegbmCV.predict_proba(X_test)[:,1]
probas_train = modelegbmCV.predict_proba(X_train)[:,1]


In [ ]:
print(roc_auc_score(y_train,probas_train))
print(roc_auc_score(y_test,probas_test))


In [ ]:
print(classification_report(y_train,modelegbmCV.predict(X_train), target_names=target_names))
print(classification_report(y_test,modelegbmCV.predict(X_test), target_names=target_names))

In [ ]:
# AJOUTE LES RESULTATS
model='gbm heavy'
# métriques (liste de dictionnaires)
metriques.append([{'model':model,'AUC_test':round(roc_auc_score(y_test,probas_test),3),'lift at 5':lift(probas_test,X_test,y_test,p=5),'lift at 10':lift(probas_test,X_test,y_test,p=10)}])
metriques
# conclusion  le modele RF est plus predictif mais moins confiance dans sa capacité à etre robuste à moyen terme


In [ ]:
# Prediction des probabilités de 1 , array2d
probas_test = gbt_Rand.predict_proba(X_test)[:,1]
probas_train = gbt_Rand.predict_proba(X_train)[:,1]


In [ ]:
print(roc_auc_score(y_train,probas_train))
print(roc_auc_score(y_test,probas_test))


In [ ]:
print(classification_report(y_train,gbt_Rand.predict(X_train), target_names=target_names))
print(classification_report(y_test,gbt_Rand.predict(X_test), target_names=target_names))

In [ ]:
# AJOUTE LES RESULTATS
model='gbm light'
# métriques (liste de dictionnaires)
metriques.append([{'model':model,'AUC_test':round(roc_auc_score(y_test,probas_test),3),'lift at 5':lift(probas_test,X_test,y_test,p=5),'lift at 10':lift(probas_test,X_test,y_test,p=10)}])
metriques
# conclusion  le modele RF est plus predictif mais moins confiance dans sa capacité à etre robuste à moyen terme


# 4. Comparaison et conclusion 

In [ ]:
metriques